# **Web Scraing for Financial Data**




### Web Scraping from Yahoo Finance

https://finance.yahoo.com

In [148]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import re

In [238]:
def get_and_transform_data(indices):
    '''
    Function for scraping data and doing appropriate manipulations
    '''
    statements_df = pd.DataFrame()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    for i in indices:
        url_is = 'https://finance.yahoo.com/quote/' + i + '/financials?p=' + i
        page = requests.get(url_is, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
        ls = []  # Creating an empty list
        for s in soup.find_all('button'):
            s.extract()
        for l in soup.find_all(['div']):
            # Finding all data structure that is ‘div’
            ls.append(l.string)  # Adding each element one by one to the list

        new_ls = list(filter(None, ls))
        new_ls = new_ls[14:]
        is_data = list(zip(*[iter(new_ls)] * 6))

        Income_st = pd.DataFrame(is_data[0:])
        Income_st.columns = Income_st.iloc[0]  # Name columns to first row of dataframe
        Income_st = Income_st.iloc[1:, ]  # start to read 1st row
        Income_st = Income_st.T  # transpose dataframe
        Income_st.columns = Income_st.iloc[0]  # Name columns to first row of dataframe
        Income_st.drop(Income_st.index[0], inplace=True)  # Drop first index row
        Income_st.index.name = ''  # Remove the index name
        Income_st.rename(index={'ttm': '9/30/2022'},inplace=True)
        Income_st = Income_st[Income_st.columns[:-5]]  # remove last 5 irrelevant columns
        Income_st['company_ind'] = i
        Income_st.index.name = 'year'
        Income_st.reset_index(inplace=True)
        statements_df = statements_df.append(Income_st)
    return statements_df

In [ ]:
indices = ['AAPL', 'AMZN', 'MSFT']
output = get_and_transform_data(indices)
if output is not None:
    output.to_csv('data/income_statement.csv', index=False)

In [351]:
df = pd.read_csv('data/income_statement.csv', thousands=',')

In [350]:
# def custom_data(company_index, year):
#     '''
#     Function for filtering data for upcoming visualizations
#     '''
#     df_temp = df[(df['company_ind'] == company_index) & (df['year'] == year)]
    
#     df_temp.loc[:, df_temp.columns.str.contains('cost|expense', case=False)] = 0
#     return df_temp

In [349]:
# data = custom_data('AMZN', '9/30/2022')

In [326]:
data = df[(df['company_ind'] == 'AMZN') & (df['year'] == '9/30/2022')]
data.head()

,year,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,...,Interest Income,Interest Expense,Net Interest Income,EBIT,EBITDA,Reconciled Cost of Revenue,company_ind,Earnings from Equity Interest Net of Tax,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest
5,9/30/2022,502191000,437379000,64812000,51841000,12971000,-1493000,-1518000,9960000,-1378000,...,662000,2155000,-1493000,12115000,"50,693,000",437379000,AMZN,-15000.0,38578000.0,11323000.0


In [330]:
Revenue = data['Total Revenue'].iloc[0]
COGS = data['Cost of Revenue'].iloc[0]*-1
GrossProfit = data['Gross Profit'].iloc[0]
OperatingExpense = data['Operating Expense'].iloc[0]*-1
InterestExpense = data['Interest Expense'].iloc[0]*-1
OperatingIncome = data['Total Operating Income as Reported'].iloc[0]

In [348]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    name = "xxx", 
    
    measure = ["relative", "relative", "total", "relative", "relative", "total"],
    x = ["Revenue", "COGS", "Gross Profit", "Operating Expense", "Interest Expense", "Total Operating Income as Reported"],
    textposition = "outside",
    text = [Revenue/100000, COGS/100000, GrossProfit/100000, OperatingExpense/100000,
            InterestExpense/100000, OperatingIncome/100000],
    y = [Revenue,COGS,GrossProfit,OperatingExpense,InterestExpense,OperatingIncome],
    connector = {"line":{"color":"rgb(63, 63, 63)"}}, ))

,
fig.update_layout(
        width=800,
        height=1000,
        title = f"Profit and loss statement ('AMZN')",
        showlegend = True
)

fig.show()